# <p style="text-align: center;">Construction of a simplified RNA-based Knowledge Graph</p>

**Objective:** [SDM-RDFizer](https://github.com/SDM-TIB/SDM-RDFizer) is an interpreter of mapping rules that allows the transformation of (un)structured data into RDF Knowledge Graphs. In this notebook, we wish to show the use of RDFizer for the generation of a RNA-based Knowledge Graph that involves **gene-disease**, **gene-miRNA**, and **miRNA-disease** relationships. These relationships are made available from the following public repositories:
- [gene-disease](https://storage.googleapis.com/pheknowlator/current_build/data/original_data/curated_gene_disease_associations.tsv), provided by PheKnowLator
- [gene-miRNA](https://dianalab.e-ce.uth.gr/downloads/tarbase_v8_data.tar.gz), provided by TarBase <!--- (we selected 50k random rows to reduce input size) -->
- [miRNA-disease](http://watson.compbio.iupui.edu:8080/miR2Disease/download/AllEntries.txt), provided by miR2Disease

The construction of the Knowledge Graph should be compliant with the following Ontologies:
- Mondo Merged Disease Ontology ([**Mondo**](https://obofoundry.org/ontology/mondo.html)), a semi-automatically constructed Ontology that merges in multiple disease resources
- Relations Ontology ([**RO**](https://obofoundry.org/ontology/ro.html)), a collection of OWL relations (ObjectProperties) intended for use across a wide variety of biological ontologies

To achieve this task, we need to map objects in RDF triples by means of RDFizer facilities, we will consider:
- [disease-mondo-map](https://storage.googleapis.com/pheknowlator/current_build/data/processed_data/DISEASE_MONDO_MAP.txt), provided by PheKnowLator
- [mirnaLabel-mirbase-map](#genemirna), obtained from [miRBase](https://mirbase.org/)
- [doid-mondo-map](#mirnadis), obtained by means of PheKnowLator ecosystem's functions

All the data required for the execution of this notebook can be downloaded from this [testRNA-KG/RDFizer/files](https://drive.google.com/drive/u/3/folders/1e54vON8b82FpqcNGesiqiVORD3vV_z2R) Google Drive repository.

**Authors:** [ECavalleri](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=emanuele.cavalleri@unimi.it), [MMesiti](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=marco.mesiti@unimi.it)

**GitHub Repositories:** [testRNA-KG/RDFizer](https://github.com/emanuelecavalleri/testRNA-KG/tree/main/RDFizer_comparison), [PheKnowLator](https://github.com/callahantiff/PheKnowLator/) (PKT) 
<!--- **Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)** --->
  
<br>  

<a target="_blank" href="https://private-user-images.githubusercontent.com/33032169/298911261-e6228042-137d-4241-8971-e674fe41d4f3.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3MDcxNzEwMTQsIm5iZiI6MTcwNzE3MDcxNCwicGF0aCI6Ii8zMzAzMjE2OS8yOTg5MTEyNjEtZTYyMjgwNDItMTM3ZC00MjQxLTg5NzEtZTY3NGZlNDFkNGYzLnBuZz9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFWQ09EWUxTQTUzUFFLNFpBJTJGMjAyNDAyMDUlMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjQwMjA1VDIyMDUxNFomWC1BbXotRXhwaXJlcz0zMDAmWC1BbXotU2lnbmF0dXJlPTgwMzA1MDAxODY4MWIxMjM4YTVmNDExYzQzMTk2YzFhODIzMDdmZjJhZjI5ZDU0NjFkZDdkYmRkZGI4NmMzN2MmWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0JmFjdG9yX2lkPTAma2V5X2lkPTAmcmVwb19pZD0wIn0.knrkWr6nevh4TrujUVwmC3CwNlYAUBYfAt5-lZDtsMA"> <img src="https://private-user-images.githubusercontent.com/33032169/298911261-e6228042-137d-4241-8971-e674fe41d4f3.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3MDcxNzEwMTQsIm5iZiI6MTcwNzE3MDcxNCwicGF0aCI6Ii8zMzAzMjE2OS8yOTg5MTEyNjEtZTYyMjgwNDItMTM3ZC00MjQxLTg5NzEtZTY3NGZlNDFkNGYzLnBuZz9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFWQ09EWUxTQTUzUFFLNFpBJTJGMjAyNDAyMDUlMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjQwMjA1VDIyMDUxNFomWC1BbXotRXhwaXJlcz0zMDAmWC1BbXotU2lnbmF0dXJlPTgwMzA1MDAxODY4MWIxMjM4YTVmNDExYzQzMTk2YzFhODIzMDdmZjJhZjI5ZDU0NjFkZDdkYmRkZGI4NmMzN2MmWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0JmFjdG9yX2lkPTAma2V5X2lkPTAmcmVwb19pZD0wIn0.knrkWr6nevh4TrujUVwmC3CwNlYAUBYfAt5-lZDtsMA"></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

<br>

***
***

## Download, load, and map data
This section downloads and loads in several different types of data, which are needed to construct the Knowledge Graph. Identifiers have to be mapped to be compliant to Mondo and miRBase. 

***
### gene-disease from [PheKnowLator](https://github.com/callahantiff/PheKnowLator)
[PheKnowLator](https://github.com/callahantiff/PheKnowLator) is a tool developed to download, process, map, and clean data in order to build edges for a Knowledge Graph. It provides you a Human Disease Molecular Mechanisms Knowledge Graph that includes gene-disease associations.

In [1]:
import pandas as pd

gene_disease = pd.read_csv("https://storage.googleapis.com/pheknowlator/current_build/"+
                            "data/original_data/curated_gene_disease_associations.tsv", sep='\t')
# The original data is filtered such that only records meeting the following criteria were included:
# EI >= 1.0 (90th percentile)
gene_disease = gene_disease[gene_disease['EI']>=1.0]
gene_disease

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.700,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.30,1.0,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.700,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.0,2015.0,2015.0,1,0,CTD_human
3,2,A2M,0.529,0.769,C0007102,Malignant tumor of colon,disease,C06;C04,Neoplastic Process,0.31,1.0,2004.0,2019.0,1,0,CTD_human
4,2,A2M,0.529,0.769,C0009375,Colonic Neoplasms,group,C06;C04,Neoplastic Process,0.30,1.0,2004.0,2004.0,1,0,CTD_human
5,2,A2M,0.529,0.769,C0011265,Presenile dementia,disease,C10;F03,Mental or Behavioral Dysfunction,0.30,1.0,1998.0,2004.0,3,0,CTD_human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84027,106480868,RN7SKP114,0.931,0.077,C4722327,"PROSTATE CANCER, HEREDITARY, 1",disease,C04;C12,Neoplastic Process,0.30,1.0,2018.0,2018.0,1,0,CTD_human
84028,106481323,RNU6-456P,0.931,0.077,C2931456,"Prostate cancer, familial",disease,C04;C12,Neoplastic Process,0.30,1.0,2018.0,2018.0,1,0,CTD_human
84029,106481323,RNU6-456P,0.931,0.077,C4722327,"PROSTATE CANCER, HEREDITARY, 1",disease,C04;C12,Neoplastic Process,0.30,1.0,2018.0,2018.0,1,0,CTD_human
84031,107075310,MTCO2P12,0.368,0.962,C0268237,Cytochrome-c Oxidase Deficiency,disease,C16;C18,Disease or Syndrome; Congenital Abnormality,0.33,1.0,1999.0,2011.0,0,0,GENOMICS_ENGLAND


In [2]:
gene_disease[['geneId','diseaseId']].to_csv(
    'files/gene-disease.csv', index=None, header=['gene', 'disease'])

gene_disease_map = pd.read_csv("https://storage.googleapis.com/pheknowlator/current_build/"+
                                "data/processed_data/DISEASE_MONDO_MAP.txt", sep='\t', header=None)
gene_disease_map.to_csv(
    'files/DISEASE_MONDO_MAP.csv', index=None, header=['umls', 'disease'])

***
### gene-miRNA from [TarBase](https://dianalab.e-ce.uth.gr/html/diana/web/index.php?r=tarbasev8/index)<a class="anchor" id="genemirna"></a>
DIANA-TarBase v8 is a reference database devoted to the indexing of experimentally supported microRNA (miRNA) targets.

In [3]:
data = pd.read_csv('https://dianalab.e-ce.uth.gr/downloads/tarbase_v8_data.tar.gz',
                         compression='gzip', sep="\t", dtype={"cell_line": "string"})
data

,TarBase_v8_download.txt,geneName,mirna,species,cell_line,tissue,category,method,positive_negative,direct_indirect,up_down,condition
0,0910001A06Rik(mmu),0910001A06Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,UP,NaN
1,1200004M23Rik(mmu),1200004M23Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,DOWN,NaN
2,1700027J05Rik(mmu),1700027J05Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,UP,NaN
3,1810015A11Rik(mmu),1810015A11Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,DOWN,NaN
4,2310047A01Rik(mmu),2310047A01Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,DOWN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
927115,vimentin(hsa),vimentin(hsa),hsa-miR-9-5p,Homo sapiens,<NA>,NaN,Cancer/Malignant,Western Blot,NEGATIVE,INDIRECT,NaN,NaN
927116,Â Â Â Â (PTPRG)(hsa),Â Â Â Â (PTPRG)(hsa),hsa-miR-146a-5p,Homo sapiens,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,UP,NaN
927117,Î±-actin(mmu),Î±-actin(mmu),mmu-miR-24-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,qPCR,POSITIVE,INDIRECT,UP,NaN
927118,Î±-actin(mmu),Î±-actin(mmu),mmu-miR-24-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Western Blot,POSITIVE,INDIRECT,UP,NaN


In [4]:
# For the time being, we keep only Homo sapiens rows
data = data.dropna()[data.dropna()['species'].str.contains("Homo sapiens")]

# Moreover, we keep only 50k (random) rows to reduce input size
#data = data.sample(n=50000)

# This simplified KG ignores if a transcript is "3p" or "5p", so we store this information as additional column
data['p'] = data[data['mirna'].str.contains("p")]['mirna']
data["p"] = data["p"].str[-2:]
data['mirna'] = data['mirna'].str.replace(r'-[35]p$', '', regex=True)
data['mirna'] = data['mirna'].str.lower()

# If you're interested in understanding why Homo sapiens has -3p and -5p miRNAs:
# https://pubmed.ncbi.nlm.nih.gov/12592000/
# Putting aside the -(3/5)p information, we are essentially dealing with non-mature (aka hairpin) miRNA:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3616697/

from pkt_kg.utils import data_downloader
import gffpandas.gffpandas as gffpd
import os
data_downloader('https://www.mirbase.org/download/hsa.gff3', '')

miRBaseMap = gffpd.read_gff3('hsa.gff3')  
os.remove('hsa.gff3')
print(miRBaseMap.header)
print(miRBaseMap.df)
miRBaseMap = miRBaseMap.attributes_to_columns()
miRBaseMap = miRBaseMap[['attributes']]
miRBaseMap
miRBaseMap = miRBaseMap.attributes.str.split(';',expand=True)
# Keep only "ID" and "Name" columns
miRBaseMap = miRBaseMap[[2,0]]
# Remove substring "ID="
miRBaseMap[0] = miRBaseMap[0].str[3:]
# Remove substring "Name="
miRBaseMap[2] = miRBaseMap[2].str[5:]
# Keep only hairpin/stem-loop miRNAs
# (those starting with MI and not MIMAT, last one is reserved for mature sequences)
miRBaseMap = miRBaseMap[~miRBaseMap[0].str.startswith('MIMAT')]
miRBaseMap.to_csv('files/MIRBASE_ID_ACCESSION_MAP.csv', header=['mirna','mirbase'], index=None)

gene_entrez_map = pd.read_csv('https://storage.googleapis.com/pheknowlator/current_build/'+
                               'data/processed_data/ENSEMBL_GENE_ENTREZ_GENE_MAP.txt', sep='\t', header=None)

data.merge(gene_entrez_map, left_on=['TarBase_v8_download.txt'], right_on=[0])[[1,'mirna']].to_csv('files/gene-miRNA.csv', index=None, header=['gene', 'mirna'])

##gff-version 3
##date 2018-3-5
#
# Chromosomal coordinates of Homo sapiens microRNAs
# microRNAs:               miRBase v22
# genome-build-id:         GRCh38
# genome-build-accession:  NCBI_Assembly:GCA_000001405.15
#
# Hairpin precursor sequences have type "miRNA_primary_transcript". 
# Note, these sequences do not represent the full primary transcript, 
# rather a predicted stem-loop portion that includes the precursor 
# miRNA. Mature sequences have type "miRNA".
#

     seq_id source                      type     start       end score strand   
0      chr1      .  miRNA_primary_transcript     17369     17436     .      -  \
1      chr1      .                     miRNA     17409     17431     .      -   
2      chr1      .                     miRNA     17369     17391     .      -   
3      chr1      .  miRNA_primary_transcript     30366     30503     .      +   
4      chr1      .                     miRNA     30438     30458     .      +   
...     ...    ...                     

***
### miRNA-disease from [miR2Disease](http://watson.compbio.iupui.edu:8080/miR2Disease/)<a class="anchor" id="mirnadis"></a>
miR2Disease, a manually curated database, aims at providing a comprehensive resource of miRNA deregulation in various human diseases.

In [5]:
mirna_disease = pd.read_csv('http://watson.compbio.iupui.edu:8080/miR2Disease/download/AllEntries.txt', sep="\t", header=None)
mirna_disease[0] = mirna_disease[0].str.lower()

descDOmap = pd.read_csv('http://watson.compbio.iupui.edu:8080/miR2Disease/download/diseaseList.txt', sep="\t")
descDOmap.columns = ['desc', 'doid']
descDOmap['doid'] = descDOmap['doid'].str.upper()
descDOmap['doid'] = descDOmap['doid'].str.replace(':', '_')

mirna_disease.columns = ['mirna', 'desc', 2,3,4,5]
mirna_disease = pd.merge(descDOmap, mirna_disease, on=['desc'])
mirna_disease

,desc,doid,mirna,2,3,4,5
0,acute lymphoblastic leukemia (ALL),DOID_9952,hsa-let-7e,down-regulated,microarray,2007.0,MicroRNA expression signatures accurately disc...
1,acute lymphoblastic leukemia (ALL),DOID_9952,hsa-mir-125a,down-regulated,microarray,2007.0,MicroRNA expression signatures accurately disc...
2,acute lymphoblastic leukemia (ALL),DOID_9952,hsa-mir-151*,up-regulated,microarray,2007.0,MicroRNA expression signatures accurately disc...
3,acute lymphoblastic leukemia (ALL),DOID_9952,hsa-mir-210,up-regulated,microarray,2007.0,MicroRNA expression signatures accurately disc...
4,acute lymphoblastic leukemia (ALL),DOID_9952,hsa-mir-22,down-regulated,"northern blot, qRT-PCR etc",2009.0,Gene silencing of MIR22 in acute lymphoblastic...
...,...,...,...,...,...,...,...
2619,Waldenstrom Macroglobulinemia (WM),DOID_9080,hsa-mir-542-3p,up-regulated,"Northern blot, qRT-PCR etc",2009.0,"microRNA expression in the biology, prognosis ..."
2620,Waldenstrom Macroglobulinemia (WM),DOID_9080,hsa-mir-9*,down-regulated,"Northern blot, qRT-PCR etc",2009.0,"microRNA expression in the biology, prognosis ..."
2621,Waldenstrom Macroglobulinemia (WM),DOID_9080,hsa-mir-155,up-regulated,"Northern blot, qRT-PCR etc",2009.0,"microRNA expression in the biology, prognosis ..."
2622,Waldenstrom Macroglobulinemia (WM),DOID_9080,hsa-mir-184,up-regulated,"Northern blot, qRT-PCR etc",2009.0,"microRNA expression in the biology, prognosis ..."


In [6]:
from typing import Tuple
from rdflib import Graph, URIRef

def gets_ontology_class_dbxrefs(graph: Graph) -> Tuple:
    dbx_uris: Dict = dict()
    dbx = [x for x in graph if 'hasdbxref' in str(x[1]).lower() if isinstance(x[0], URIRef)]
    for x in dbx:
        if str(x[2]).lower() in dbx_uris.keys(): dbx_uris[str(x[2]).lower()].append(str(x[0]))
        else: dbx_uris[str(x[2]).lower()] = [str(x[0])]
    dbx_type = {str(x[2]).lower(): 'DbXref' for x in dbx}

    ex_uris: Dict = dict()
    ex = [x for x in graph if 'exactmatch' in str(x[1]).lower() if isinstance([0], URIRef)]
    for x in ex:
        if str(x[2]).lower() in ex_uris.keys(): ex_uris[str(x[2]).lower()].append(str(x[0]))
        else: ex_uris[str(x[2]).lower()] = [str(x[0])]
    ex_type = {str(x[2]).lower(): 'ExactMatch' for x in ex}

    return {**dbx_uris, **ex_uris}, {**dbx_type, **ex_type}

In [7]:
mondo_graph = Graph().parse("http://purl.obolibrary.org/obo/mondo.owl", format='application/rdf+xml')
dbxref_res = gets_ontology_class_dbxrefs(mondo_graph)[0]

# Fix DOIDs (substitute : with _) and upper case them
mondo_dict = {str(k).replace(':','_').upper(): {str(i).split('/')[-1].replace(':','_') for i in v}
              for k, v in dbxref_res.items() if 'doid' in str(k)}

with open('files/DOID_MONDO_MAP.csv', 'w') as outfile:
    outfile.write("doid,disease\n")
    for k, v in mondo_dict.items():
        outfile.write(str(k) + ',' + str(v).replace('{','').replace(', ','').replace('\'','').replace('}','') + '\n')

In [8]:
mirbase_mirna_map = pd.read_csv("files/MIRBASE_ID_ACCESSION_MAP.csv")
mirna_disease.merge(mirbase_mirna_map, on=['mirna'])[['mirbase', 'doid']].to_csv('files/mirna-disease.csv', index=None, header=['mirbase', 'doid'])

***
We can copy-paste `mapping.ttl` content in [**rdf-grapher**](https://www.ldf.fi/service/rdf-grapher) to graphically visualize it.

<a target="_blank" href=https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/b0a7b4d6-85ac-427f-8034-3b2da07fed7c> <img src=https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/b0a7b4d6-85ac-427f-8034-3b2da07fed7c></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

Another visualization tool is [**rdf-visualizer**](https://issemantic.net/rdf-visualizer), which allows us to dynamically inspect the graph (in the figure below you can find a screenshot of the output).

<a target="_blank" href=https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/f9a1feaa-7381-4fb4-8e72-78d097faaddc> <img src=https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/f9a1feaa-7381-4fb4-8e72-78d097faaddc></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

In [9]:
!python3 -m rdfizer -c config.ini

Semantifying RNA-KG...
TM: file:///home/ecavalleri/testRNA-KG/notebooks/gene-miRNA
TM: file:///home/ecavalleri/testRNA-KG/notebooks/gene-disease
TM: file:///home/ecavalleri/testRNA-KG/notebooks/miRNA-disease
TM: file:///home/ecavalleri/testRNA-KG/notebooks/disease-mondo-map
TM: file:///home/ecavalleri/testRNA-KG/notebooks/mirnaLabel-mirbase-map
TM: file:///home/ecavalleri/testRNA-KG/notebooks/doid-mondo-map
Successfully semantified RNA-KG.


Successfully semantified all datasets in 25.313 seconds.


Let us consider for instance *MEGF8* gene (entrez ID [1954](http://www.ncbi.nlm.nih.gov/gene/1954)). Biologically speaking, the protein encoded by this gene is a membrane protein that contains several [EGF-like](https://www.uniprot.org/keywords/KW-0245) and [PSI](https://www.ebi.ac.uk/interpro/entry/InterPro/IPR016201/) domains. Defects in this gene are a cause of [Carpenter syndrome 2](https://www.ncbi.nlm.nih.gov/medgen/905199) ([MONDO_0013998](http://purl.obolibrary.org/obo/MONDO_0013998)).

In [10]:
from rdflib import Graph

# Read in the RNA-KG generated using RDFizer
g = Graph()
g.parse("output/RNA-KG.ttl", format='ttl')

<Graph identifier=Nee8f1de8e98e4eb4a46c0e735b5641bf (<class 'rdflib.graph.Graph'>)>

In [11]:
# Utility function to add edges to a graph

from typing import List, Set, Union

def adds_edges_to_graph(graph: Graph, edge_list: Union[List, Set], progress_bar: bool = True) -> Graph:
    
    edge_list_updated = set(edge_list) if isinstance(edge_list, List) else edge_list
    edge_set = edge_list_updated if progress_bar else edge_list_updated
    for edge in edge_set: graph.add(edge)

    return graph

In [12]:
# Obtain a Knowledge subGraph having only MEGF8 as subject

from rdflib.extras.external_graph_libs import *
from rdflib import Graph, URIRef

filtered_triples = []
for s, p, o in g:
    if str(s).startswith(str(URIRef('http://www.ncbi.nlm.nih.gov/gene/1954'))):
        filtered_triples += [(s, p, o)]

print('There are {} edges'.format(len(filtered_triples)))
sub_graph = adds_edges_to_graph(Graph(), filtered_triples)

sub_graph.serialize(destination='output/subRNA-KG.nt', format='nt', encoding='utf-8')

There are 33 edges


<Graph identifier=N65feae6a362347b9b301833770f73c66 (<class 'rdflib.graph.Graph'>)>

Below I depict 12 of them for the sake of visualization.

<a target="_blank" href=https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/ed6c1ead-03fa-4bd4-addf-010592f36a6b> <img src=https://github.com/emanuelecavalleri/testRNA-KG/assets/33032169/ed6c1ead-03fa-4bd4-addf-010592f36a6b></a> 

MEGF8 (entrez ID [1954](http://www.ncbi.nlm.nih.gov/gene/1954)):
- *is_a* (http://www.w3.org/1999/02/22-rdf-syntax-ns#type) gene ([SO_0000704](http://purl.obolibrary.org/obo/SO_0000704))
- which *causes or contributes to condition* ([RO_0003302](http://purl.obolibrary.org/obo/RO_0003302)) Carpenter syndrome ([MONDO_0013998](http://purl.obolibrary.org/obo/MONDO_0013998)),
- and *interacts with* ([RO_0002434](http://purl.obolibrary.org/obo/RO_0002434)) hsa-mir-1271 ([MI0003814](https://mirbase.org/hairpin/MI0003814)).

hsa-mir-1271 is a microRNA (miRNA) known to [target](https://en.wikipedia.org/wiki/MicroRNA#Targets) MEGF8 transcripts. If you are interested in this topic, please watch the seminar [*An introduction to the RNA world*](https://youtu.be/QjNeOWaKKEE) for further biologial information on how this targeting occurs.